# 평가(Evaluation)

## 정확도(Accuracy)

- 정확도 = 예측결과가 동일한 데이터 건수/전체 예측 데이터 건수

- scikit learn 의 BaseEstimator클래스를 상속받으면
  Customized 형태의 Estimator를 개발자가 생성가능
  
- fit() 메서드는 아무것도 수행하지 않음.
- predict() 메서드는 단순히 피처가 1이면 0 그렇치 않으면 1로 예측

In [7]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [8]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    #fit 메서드는 아무것도 학습하지 않음
    def fit(self,X,y=None):
        pass
    #predict()메서드는 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측
    def predict(self,X):
        pred=np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i]=0
            else :
                pred[i]=1
        return pred

In [10]:
# MyDummyClassfier로 학습/예측/평가 하기

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 타이타닉 데이터 로딩 및 가공/ 데이터 분리
titanic_df = pd.read_csv('titanic_train.csv')
y_df = titanic_df['Survived']
X_df = titanic_df.drop('Survived',axis=1)
X_df = transform_features(X_df)
X_train,X_test,y_train,y_test=train_test_split(X_df,y_df,test_size=0.2, random_state=0)

# DummyClassfier로 학습/예측/평가
Dummyclf = MyDummyClassifier()
Dummyclf.fit(X_train,y_train)
pred = Dummyclf.predict(X_test)

print('MyDummyClassifier 정확도 :{0:4f}'.format(accuracy_score(y_test,pred)))

MyDummyClassifier 정확도 :0.787709
